In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [2]:
cyclone_events = np.load('data/ERA5/ERA5_MSL_1982_2020_3h_0.75/cyclones_events.npz')
closeness_w = np.load('H:/Climate/data/ERA5/ERA5_MSL_1982_2020_3h_0.75/metrics_corr_land_masked_and_preproc_window_2d_delay_0d/diff_metrics/network_metrics/closeness_w.npy')
degree_w = np.load('H:/Climate/data/ERA5/ERA5_MSL_1982_2020_3h_0.75/metrics_corr_land_masked_and_preproc_window_2d_delay_0d/diff_metrics/network_metrics/degree_w.npy')
EVC_w = np.load('H:/Climate/data/ERA5/ERA5_MSL_1982_2020_3h_0.75/metrics_corr_land_masked_and_preproc_window_2d_delay_0d/diff_metrics/network_metrics/EVC_w.npy')
GCC_w = np.load('H:/Climate/data/ERA5/ERA5_MSL_1982_2020_3h_0.75/metrics_corr_land_masked_and_preproc_window_2d_delay_0d/diff_metrics/network_metrics/GCC_w.npy')
LCC_w = np.load('H:/Climate/data/ERA5/ERA5_MSL_1982_2020_3h_0.75/metrics_corr_land_masked_and_preproc_window_2d_delay_0d/diff_metrics/network_metrics/LCC_w.npy')
MSLP_preproc = np.load('H:/Climate/data/ERA5/ERA5_MSL_1982_2020_3h_0.75/metrics_corr_land_masked_and_preproc_window_2d_delay_0d/diff_metrics/input_data/MSLP_preproc.npy')

In [3]:
cyclone_events_2 = cyclone_events[cyclone_events.files[0]]
cyclone_events_4 = cyclone_events[cyclone_events.files[1]]
cyclone_events_6 = cyclone_events[cyclone_events.files[2]]
cyclone_events_8 = cyclone_events[cyclone_events.files[3]]
cyclone_events_10 = cyclone_events[cyclone_events.files[4]]
cyclone_events_12 = cyclone_events[cyclone_events.files[5]]

In [4]:
cyclone_events_2.shape

(36, 69, 113960)

In [5]:
MSLP_preproc = np.reshape(MSLP_preproc, (36, 69, 113960))

In [6]:
labels = np.zeros(113960)

for i in range(113960):
    if len(cyclone_events_2[:, :, i][cyclone_events_2[:, :, i] != False]) > 0:
        labels[i] = 1

In [208]:
'''
cyclone_occurances = []
no_cyclone = []
for i in range(113960):
    if len(cyclone_events_2[:, :, i][cyclone_events_2[:, :, i] != False]) > 0:
        cyclone_occurances.append(i)
    else:
        no_cyclone.append(i)
        
import random
random.shuffle(cyclone_occurances)
random.shuffle(no_cyclone)    

f1 = open("shuffle_cyclone.csv", "w")
f2 = open("shuffle_no_cyclone.csv", "w")
[f1.write(str(item) + ',') for item in cyclone_occurances]
[f2.write(str(item) + ',') for item in no_cyclone]
f1.close()
f2.close()
'''

In [41]:
cyclone_occurances = pd.read_csv("shuffle_cyclone.csv", header=None).values[0]
no_cyclone = pd.read_csv("shuffle_no_cyclone.csv", header=None).values[0]

cut_cyclone = int(0.8 * len(cyclone_occurances))
cut_no_cyclone = int(0.8 * len(no_cyclone))

train_id = cyclone_occurances[:cut_cyclone].astype(int)
train_id = np.append(train_id, no_cyclone[:cut_no_cyclone].astype(int))

test_id = cyclone_occurances[cut_cyclone:-1].astype(int) 
test_id = np.append(test_id, no_cyclone[cut_no_cyclone:-1].astype(int))

train_data = torch.tensor(MSLP_preproc[:, :, train_id], dtype=torch.double)
test_data = torch.tensor(MSLP_preproc[:, :, test_id], dtype=torch.double)

labels_train = torch.tensor(labels[train_id])
labels_test = torch.tensor(labels[test_id])

In [42]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()        
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=36,            
                kernel_size=(3, 3),          ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(36, 72, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )        
        
        # fully connected layer, output 10 classes
        self.out = nn.Linear(72 * 36 * 69, 2)    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)        
        
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    
        

In [43]:
cnn = CNN()
loss_func = nn.CrossEntropyLoss()  
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)  

In [44]:
train_data[:, :, 0]

tensor([[  0.0000,   0.0000,      nan,  ...,      nan,      nan,      nan],
        [     nan,      nan,      nan,  ...,      nan,      nan,      nan],
        [     nan,      nan,      nan,  ...,      nan,      nan,      nan],
        ...,
        [     nan,      nan,  -8.0249,  ..., -64.3642, -78.0519,      nan],
        [     nan,      nan,  -9.1256,  ...,  -3.7856, -73.8378,      nan],
        [     nan,      nan,  -9.8328,  ...,      nan, -26.1540, -17.9396]],
       dtype=torch.float64)

In [47]:
num_epochs = 10

def train(num_epochs, cnn, train_data, labels_train):  
    cnn.train()
    
    cnn = cnn.double()
        
    # Train the model
    batch_size = int(0.1 * train_data.shape[2])
        
    for epoch in range(num_epochs):
        for i in range(10):
            
            image = train_data[:, :, (i * batch_size) : (i + 1) * batch_size]
            label = labels_train[i*batch_size : (i+1) * batch_size]
            
            image = image.unsqueeze(1) 
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(image)   # batch x
            b_y = Variable(label)   # batch y
            output = cnn(b_x)[0]               
            loss = loss_func(output, b_y)

            # clear gradients for this training step   
            optimizer.zero_grad()           

            # backpropagation, compute gradients 
            loss.backward()                # apply gradients             
            optimizer.step()  
        pass
    pass

train(num_epochs, cnn, train_data, labels_train)

RuntimeError: [enforce fail at C:\Users\builder\tkoch\workspace\pytorch\pytorch_1647970138273\work\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 38978755200 bytes.